In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px

In [26]:
diabetes = pd.read_csv('data/diabetes_data.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [53]:
dupl_columns = list(diabetes.columns)

#mask = diabetes.duplicated(subset=dupl_columns)
#diabetes_dupl = diabetes[mask]

diabetes = diabetes.drop_duplicates(subset=dupl_columns)
print(diabetes.shape[0])

768


In [54]:
def low_informative_feature(data):
    low_inf_cols = []
    for col in data.columns:
        top_freq = data[col].value_counts(normalize=True).max()
        nunique_ratio = data[col].nunique() / data[col].count()
    
        if (top_freq > 0.95) and (nunique_ratio > 0.95):
            low_inf_cols.append(col)
    return low_inf_cols

print(low_inf_cols)

['Gender']


In [55]:
diabetes = diabetes.drop(low_inf_cols, axis=1)
display(diabetes.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 60.0 KB


None

In [56]:
diabetes['Glucose'] = diabetes['Glucose'].apply(lambda x: np.nan if x == 0 else x)
diabetes['BloodPressure'] = diabetes['BloodPressure'].apply(lambda x: np.nan if x == 0 else x)
diabetes['SkinThickness'] = diabetes['SkinThickness'].apply(lambda x: np.nan if x == 0 else x)
diabetes['Insulin'] = diabetes['Insulin'].apply(lambda x: np.nan if x == 0 else x)
diabetes['BMI'] = diabetes['BMI'].apply(lambda x: np.nan if x == 0 else x)

In [57]:
round(diabetes['Insulin'].isnull().mean(), 2)

0.49

In [58]:
thresh = diabetes.shape[0] * 0.7
diabetes = diabetes.dropna(how='any', thresh=thresh, axis=1)
display(diabetes.shape[1])

8

In [66]:
thresh = diabetes.shape[1] - 2
diabetes = diabetes.dropna(how='any', thresh=thresh, axis=0)
display(diabetes.shape[0])

761

In [68]:
diabetes.isnull().mean()

Pregnancies                 0.000000
Glucose                     0.006570
BloodPressure               0.036794
SkinThickness               0.289093
BMI                         0.005256
DiabetesPedigreeFunction    0.000000
Age                         0.000000
Outcome                     0.000000
dtype: float64

In [73]:
values = {
    'Glucose': diabetes['Glucose'].median(),
    'BloodPressure': diabetes['BloodPressure'].median(),
    'SkinThickness': diabetes['SkinThickness'].median(),
    'BMI': diabetes['BMI'].median()
}
inf_diabetes = diabetes.fillna(values)

In [74]:
round(diabetes['SkinThickness'].mean(), 1)

29.1

In [79]:
def outliers_iqr(data, feature, left=1.5, right=1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    q_1, q_3 = x.quantile(0.25), x.quantile(0.75)
    iqr = q_3 - q_1
    lower_bound = q_1 - left*iqr
    upper_bound = q_3 + right*iqr
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned_diabetes = outliers_iqr(diabetes, 'SkinThickness')
print(outliers.shape[0])


87


In [81]:
def outliers_z_score(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, clean_diab = outliers_z_score_mod(diabetes, 'SkinThickness')
print(outliers.shape[0])


4


In [89]:
outliers, _ = outliers_iqr(clean_diab, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr(clean_diab, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0], outliers_log.shape[0])
print(outliers.shape[0] - outliers_log.shape[0])

28 0
28


In [88]:
outliers, _ = outliers_iqr(diabetes, 'DiabetesPedigreeFunction')
outliers_log, _ = outliers_iqr(diabetes, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0], outliers_log.shape[0])
print(outliers.shape[0] - outliers_log.shape[0])

29 0
29
